### SDC Profile

### What features are driving student success in SDC application?

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
question = pd.read_csv('admissions questions-nd013.csv')

In [16]:
pd.pivot_table(question, values='question_id', index=['cohort_id'],columns=['tag'], aggfunc=np.unique).T

cohort_id,7,18,47,88,89,90,220,293,359,389
tag,,,,,,,,,,
C,8afe0061-746b-4bd7-807c-393fe5c7599d,None,None,None,None,None,None,None,None,None
C++,None,7a09e78a-8000-459e-9d79-aa21db135138,7a09e78a-8000-459e-9d79-aa21db135138,7a09e78a-8000-459e-9d79-aa21db135138,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5
age,1bfe0061-746b-4bd7-807c-393fe5c7599d,None,None,None,None,None,None,None,None,None
degree relevant,None,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4
education,None,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae
education institution,None,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224
employer,None,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370
employment,None,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5
gender,None,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c


### Exclude cohort 7
### Hypothesis Tests on Topics:
* Gender
* Degree relevant
* Education
* Education institution
* Professional experience
* Employer
* Employment
* Python
* C++
* Programming skills
* Theory skills
* Time commitment
* GitHub / linkedin account
* Goal
* Purpose
* Phone
* Location

In [ ]:
laurel = open('conn_laurel.txt', 'r')
conn_laurel = psycopg2.connect(laurel.read())

In [17]:
3883+3585

7468

In [18]:
11325-215-3640

7470

In [19]:
1432/251986

0.005682855396728389

In [20]:
1229/281539

0.0043652921975285836

In [21]:
1122/257663

0.004354525096734882

In [22]:
1430/260918

0.005480649092818433

In [25]:
2284/260918

0.008753708061536575

In [26]:
1252/165085

0.0075839718932671045